In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [6]:
df1 = pd.read_csv("data\engineered_application_train.csv")
df1.shape

<>:1: SyntaxWarning: invalid escape sequence '\e'
<>:1: SyntaxWarning: invalid escape sequence '\e'
C:\Users\mudga\AppData\Local\Temp\ipykernel_19580\700031033.py:1: SyntaxWarning: invalid escape sequence '\e'
  df1 = pd.read_csv("data\engineered_application_train.csv")


(307511, 178)